In [1]:
from Prepare_Data import *
from Models import *
from Test import *
import os 
if os.path.isdir("scalers")==False:
    os.mkdir("scalers")
if os.path.isdir("models")==False:
    os.mkdir("models")


Using TensorFlow backend.


In [2]:
modelNumber=1
# 1: Basic LSTM Model 
# 2: Input Features first goes throush encoder, Then given to LSTM. 
# 3: Image CNN LSTM
features=2
# features= 0:No Features,1:Support Points,2:ShapeContext,3:AutoEncoder,4:Image
FeatureSize=110
# FeatureSize= No Feature:0,Support Points:4, ShapeContext:110,420,1640,AutoEncoder:256,512,1024 Image:(128,128,1)


In [3]:
## Model Parameters:
seq_length=15
n_neurons=128

In [4]:
if modelNumber==1:
    model = model_type_1(n_neurons,seq_length, FeatureSize+6)
elif modelNumber==2:
    model = model_type_2(n_neurons,seq_length, FeatureSize+6)
elif modelNumber==3:
    model = model_type_3(n_neurons,seq_length, FeatureSize)

# Training

In [5]:
from IPython.display import clear_output
error=100
if modelNumber==3:
    TrainDg,_=prepare_data_for_model(modelNumber,features,FeatureSize,seq_length)
    for epoch in range(50):
        model.fit_generator(generator=TrainDg,
                            steps_per_epoch=300,
                            epochs=1,
                            use_multiprocessing=True,
                            verbose=1,
                            workers=4)
        clear_output()
        print 'EPOCH:',epoch
        print '----------------'
        val_error,var_error_std=validate_model(modelNumber,features,FeatureSize,seq_length,model)
        if val_error<error:
            error=val_error
            print 'copied'
            model_copy = keras.models.clone_model(model)
            model_copy.set_weights(model.get_weights())
    model_copy.save('models/m_'+str(modelNumber)+'_f'+str(features)+'.h5')
else:
    x,y = prepare_data_for_model(modelNumber,features,FeatureSize,seq_length)
    for epoch in range(50):
        model.fit(x,y,epochs=1)
        clear_output()
        print 'EPOCH:',epoch
        print '----------------'
        val_error,var_error_std=validate_model(modelNumber,features,FeatureSize,seq_length,model)
        if val_error<error:
            error=val_error
            print 'copied'
            model_copy = keras.models.clone_model(model)
            model_copy.set_weights(model.get_weights())
    model_copy.save('models/m_'+str(modelNumber)+'_f'+str(features)+'_fs'+str(FeatureSize)+'.h5')

EPOCH: 49
----------------
Total XYZ MSE Error Mean=5.04 Std=4.03


# Test

In [6]:
test_error,test_error_std,real_trajectories,predicted_trajectories=test_model(modelNumber,features,FeatureSize,seq_length,model_copy)

Total XYZ MSE Error Mean=5.32 Std=4.60
